In [1]:
import numpy as np
from steam_review_scraper import get_game_review 
import timeit

start_time = timeit.default_timer()

reviews_df = get_game_review(1680880, language='english')
print(reviews_df.shape)

r_df = reviews_df[reviews_df['recommend'] == "Recommended"]
print(r_df.shape)

nr_df = reviews_df[reviews_df['recommend'] != "Recommended"]
print(nr_df.shape)

elapsed_time = timeit.default_timer() - start_time
print(f"Total Elapsed time: {elapsed_time:.2f} seconds")

Total Elapsed time: 100.29 seconds


In [26]:
print(reviews_df.shape)
r_df = reviews_df[reviews_df['recommend'] == "Recommended"]
print(r_df.shape)

nr_df = reviews_df[reviews_df['recommend'] != "Recommended"]
print(nr_df.shape)

(3270, 8)
(2021, 8)
(1249, 8)
(2021, 8, 1249, 8)


In [27]:
reviews_df.review

0       Okay. I'm almost done with the game and I feel...
1       Because of this game I will never pre-order a ...
2       As of writing I have put 14 hours into this ga...
3       I tried REALLY hard to like this game. I reall...
4       Cringe dialogue, unlikable protagonist, empty ...
                              ...                        
3265    I have a fully built gaming PC that's just a f...
3266    I have between the recommended and minimum req...
3267    Despite the hefty spec requirements, there was...
3268               Love it runs very good on a modern pc.
3269                         A magical world without mana
Name: review, Length: 3270, dtype: object

In [29]:
reviews_df.review

0       Okay. I'm almost done with the game and I feel...
1       Because of this game I will never pre-order a ...
2       As of writing I have put 14 hours into this ga...
3       I tried REALLY hard to like this game. I reall...
4       Cringe dialogue, unlikable protagonist, empty ...
                              ...                        
2285                     Excellent game play, lots of fun
2286    Right trigger, B, Right trigger, Right trigger...
2287    reviews are misleading. Dialogue is fine and t...
2288                                                  meh
2289    Super good, I like it alot so far. Nice to see...
Name: review, Length: 2290, dtype: object

In [3]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from pprint import pprint
import os
import pyLDAvis.gensim_models as gensimvis
import pickle
import pyLDAvis

stop_words = stopwords.words('english')
# stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tbont\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [44]:
import gensim.corpora as corpora
def show_LDAvis(dataframe):
    data = dataframe.review.values.tolist()
    data_words = list(sent_to_words(data))
    data_words = remove_stopwords(data_words)
    
    id2word = corpora.Dictionary(data_words)
    texts = data_words
    corpus = [id2word.doc2bow(text) for text in texts]
    
    #LDA model training
    num_topics = 10
    lda_model = gensim.models.LdaMulticore(corpus=corpus, id2word=id2word, num_topics=num_topics)
    pprint(lda_model.print_topics())
    doc_lda = lda_model[corpus]
    
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
    return LDAvis_prepared

In [45]:
total_LDAvis_prepared = show_LDAvis(reviews_df)
total_LDAvis_prepared

[(0,
  '0.040*"game" + 0.011*"like" + 0.008*"story" + 0.008*"world" + '
  '0.006*"combat" + 0.005*"get" + 0.005*"fun" + 0.005*"people" + 0.005*"bad" + '
  '0.004*"time"'),
 (1,
  '0.032*"game" + 0.010*"fun" + 0.010*"like" + 0.010*"combat" + 0.009*"story" '
  '+ 0.008*"world" + 0.006*"bad" + 0.006*"even" + 0.006*"character" + '
  '0.006*"get"'),
 (2,
  '0.036*"game" + 0.014*"like" + 0.009*"story" + 0.008*"world" + 0.008*"would" '
  '+ 0.007*"good" + 0.007*"fun" + 0.007*"really" + 0.007*"get" + '
  '0.006*"combat"'),
 (3,
  '0.023*"game" + 0.015*"like" + 0.010*"story" + 0.007*"fun" + 0.007*"combat" '
  '+ 0.007*"get" + 0.007*"lot" + 0.007*"good" + 0.006*"world" + '
  '0.005*"really"'),
 (4,
  '0.044*"game" + 0.013*"really" + 0.013*"like" + 0.010*"story" + '
  '0.009*"combat" + 0.007*"get" + 0.007*"play" + 0.006*"world" + 0.006*"even" '
  '+ 0.006*"fun"'),
 (5,
  '0.053*"game" + 0.009*"story" + 0.008*"world" + 0.007*"like" + '
  '0.006*"combat" + 0.006*"play" + 0.006*"get" + 0.005*"good" 

C:\Users\tbont\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.012151  0.003348       1        1  31.417741
4     -0.009806  0.006052       2        1  15.005001
1     -0.008107  0.001049       3        1  10.647660
6     -0.005494 -0.007012       4        1   9.852842
5     -0.004944  0.003375       5        1   9.254228
2     -0.002043 -0.004434       6        1   6.661677
3      0.003042 -0.007054       7        1   6.312529
0      0.001235 -0.001707       8        1   6.257413
8      0.019660 -0.016686       9        1   2.672282
9      0.018608  0.023068      10        1   1.918628, topic_info=        Term         Freq        Total Category  logprob  loglift
54      game  8176.000000  8176.000000  Default  30.0000  30.0000
84      like  2396.000000  2396.000000  Default  29.0000  29.0000
66      good  1403.000000  1403.000000  Default  28.0000  28.0000
126   really  1348.000000  1348.000000  Default  27.0000  27.0000
372      fun  1348.000000  1348.000000  Default  26.0000  26.0000
...      ...          ...          ...      ...      ...      ...
56     games    14.238707   808.366231  Topic10  -5.6080  -0.0855
1192     fps    11.318864   595.169865  Topic10  -5.8375  -0.0088
420    still    10.565608   563.398467  Topic10  -5.9064  -0.0228
550      far    10.804666   705.090214  Topic10  -5.8840  -0.2248
271    magic    10.692321   675.685570  Topic10  -5.8945  -0.1926

[782 rows x 6 columns], token_table=       Topic      Freq            Term
term                                  
12550      3  0.568481  actingnegative
2          1  0.308351        actually
2          2  0.136769        actually
2          3  0.109415        actually
2          4  0.077088        actually
...      ...       ...             ...
9803       5  0.604716  第二个是游戏需要更新更多东西
12132      8  0.650773              ｰr
12063      6  0.442881        𝗛𝗶𝘀𝗽𝗮𝗻𝗶𝗰
12078      6  0.442397              𝗼𝗻
12085      6  0.442881            𝘃𝗲𝗿𝘆

[1730 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 5, 2, 7, 6, 3, 4, 1, 9, 10])

In [51]:
pyLDAvis.display(total_LDAvis_prepared)

AttributeError: 'PreparedData' object has no attribute 'show'

In [39]:
r_df_LDAvis_prepared = show_LDAvis(r_df)
r_df_LDAvis_prepared

[(0,
  '0.032*"game" + 0.012*"story" + 0.011*"like" + 0.006*"would" + 0.006*"world" '
  '+ 0.006*"good" + 0.006*"fun" + 0.005*"combat" + 0.005*"time" + '
  '0.005*"people"'),
 (1,
  '0.023*"game" + 0.013*"like" + 0.012*"story" + 0.011*"really" + '
  '0.010*"combat" + 0.007*"fun" + 0.007*"world" + 0.006*"games" + 0.006*"lot" '
  '+ 0.005*"get"'),
 (2,
  '0.040*"game" + 0.015*"story" + 0.010*"like" + 0.009*"world" + '
  '0.007*"combat" + 0.007*"really" + 0.007*"get" + 0.006*"good" + '
  '0.006*"magic" + 0.005*"play"'),
 (3,
  '0.040*"game" + 0.015*"world" + 0.014*"story" + 0.010*"like" + 0.010*"fun" + '
  '0.008*"really" + 0.008*"combat" + 0.007*"good" + 0.006*"open" + '
  '0.005*"would"'),
 (4,
  '0.042*"game" + 0.012*"like" + 0.011*"fun" + 0.011*"story" + 0.010*"good" + '
  '0.008*"combat" + 0.008*"world" + 0.008*"get" + 0.007*"bad" + '
  '0.006*"really"'),
 (5,
  '0.035*"game" + 0.012*"story" + 0.011*"really" + 0.009*"fun" + 0.008*"like" '
  '+ 0.008*"combat" + 0.006*"good" + 0.006*"t

C:\Users\tbont\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.015577  0.005911       1        1  23.902302
3     -0.012257 -0.001344       2        1  20.819289
2     -0.009069 -0.004432       3        1  19.109309
6     -0.008512  0.006481       4        1  17.922134
5     -0.000314 -0.011938       5        1   4.965560
7      0.005862 -0.000521       6        1   3.598923
1      0.004312 -0.017842       7        1   3.438273
8      0.005755  0.008663       8        1   2.629015
9      0.004469  0.012634       9        1   2.237644
0      0.025330  0.002388      10        1   1.377551, topic_info=           Term         Freq        Total Category  logprob  loglift
145        game  5353.000000  5353.000000  Default  30.0000  30.0000
350       story  1559.000000  1559.000000  Default  29.0000  29.0000
206        like  1463.000000  1463.000000  Default  28.0000  28.0000
603      really   903.000000   903.000000  Default  27.0000  27.0000
60       combat  1033.000000  1033.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
1062      think     5.159464   452.536698  Topic10  -5.8745  -0.1892
425        also     5.233519   485.389061  Topic10  -5.8602  -0.2450
52    character     5.047197   443.884663  Topic10  -5.8965  -0.1919
256         one     5.006642   461.631257  Topic10  -5.9046  -0.2391
517       great     5.013659   591.408264  Topic10  -5.9032  -0.4855

[785 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
9808      1  0.494831  absolution
1330      1  0.223831      acting
1330      2  0.321757      acting
1330      3  0.174868      acting
1330      4  0.153884      acting
...     ...       ...         ...
9564      1  0.494811        ヽl__
9565      1  0.494837          ｰr
9486     10  0.777991    𝗛𝗶𝘀𝗽𝗮𝗻𝗶𝗰
9501     10  0.778121          𝗼𝗻
9508     10  0.778011        𝘃𝗲𝗿𝘆

[1662 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 3, 7, 6, 8, 2, 9, 10, 1])

In [40]:
nr_df_LDAvis_prepared = show_LDAvis(nr_df)
nr_df_LDAvis_prepared

[(0,
  '0.030*"game" + 0.007*"like" + 0.007*"really" + 0.007*"even" + 0.006*"story" '
  '+ 0.006*"character" + 0.006*"bad" + 0.006*"much" + 0.005*"world" + '
  '0.005*"fps"'),
 (1,
  '0.012*"game" + 0.006*"world" + 0.005*"like" + 0.005*"one" + 0.005*"story" + '
  '0.004*"combat" + 0.004*"character" + 0.004*"bad" + 0.004*"even" + '
  '0.003*"time"'),
 (2,
  '0.049*"game" + 0.014*"like" + 0.012*"story" + 0.008*"play" + 0.008*"bad" + '
  '0.007*"really" + 0.007*"get" + 0.005*"world" + 0.005*"even" + 0.005*"time"'),
 (3,
  '0.033*"game" + 0.012*"like" + 0.009*"even" + 0.006*"world" + 0.006*"combat" '
  '+ 0.006*"really" + 0.006*"get" + 0.005*"story" + 0.004*"would" + '
  '0.004*"play"'),
 (4,
  '0.023*"game" + 0.007*"like" + 0.007*"world" + 0.005*"combat" + 0.005*"even" '
  '+ 0.005*"story" + 0.004*"get" + 0.004*"character" + 0.004*"bad" + '
  '0.004*"hours"'),
 (5,
  '0.042*"game" + 0.015*"like" + 0.009*"get" + 0.008*"would" + 0.007*"really" '
  '+ 0.006*"combat" + 0.006*"story" + 0.006*"

C:\Users\tbont\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.031536  0.009138       1        1  36.901865
5     -0.028115  0.000297       2        1  25.151673
6     -0.021066  0.010619       3        1  11.161903
7     -0.019786 -0.017647       4        1  10.878277
9     -0.009144 -0.015082       5        1   7.019986
3     -0.001930  0.007150       6        1   3.903381
4      0.019916 -0.019402       7        1   1.563058
0      0.013557  0.007925       8        1   1.458680
8      0.023041  0.019058       9        1   1.157764
1      0.055062 -0.002056      10        1   0.803413, topic_info=         Term         Freq        Total Category  logprob  loglift
54       game  2960.000000  2960.000000  Default  30.0000  30.0000
84       like   887.000000   887.000000  Default  29.0000  29.0000
242      even   465.000000   465.000000  Default  28.0000  28.0000
195     world   357.000000   357.000000  Default  27.0000  27.0000
148     story   558.000000   558.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
66       good     1.403269   348.137683  Topic10  -5.9795  -0.6897
126    really     1.445749   435.167303  Topic10  -5.9497  -0.8830
250     first     1.249966   223.109146  Topic10  -6.0952  -0.3605
340     would     1.302753   350.730453  Topic10  -6.0538  -0.7715
55   gameplay     1.156109   239.279243  Topic10  -6.1733  -0.5085

[777 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1187      4  0.515249     acted
2         1  0.358114  actually
2         2  0.236203  actually
2         3  0.099053  actually
2         4  0.091433  actually
...     ...       ...       ...
342       8  0.012708   writing
342       9  0.012708   writing
342      10  0.012708   writing
7041      4  0.669341        ye
6854      5  0.738388    zalost

[1583 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 6, 7, 8, 10, 4, 5, 1, 9, 2])

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.014696 -0.003827       1        1  27.196225
0      0.010557  0.006562       2        1  17.934745
1      0.007384  0.004084       3        1  15.577427
9      0.005864  0.000192       4        1  12.662979
5     -0.005728 -0.003965       5        1   6.134915
2      0.007755 -0.008376       6        1   5.376363
6     -0.008874  0.013185       7        1   5.019107
4     -0.006699 -0.015616       8        1   4.839545
8     -0.006497  0.012837       9        1   2.806802
7     -0.018458 -0.005077      10        1   2.451892, topic_info=         Term         Freq        Total Category  logprob  loglift
54       game  8510.000000  8510.000000  Default  30.0000  30.0000
148     story  2053.000000  2053.000000  Default  29.0000  29.0000
84       like  2379.000000  2379.000000  Default  28.0000  28.0000
372       fun  1340.000000  1340.000000  Default  27.0000  27.0000
66       good  1302.000000  1302.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
242      even    16.462196  1013.802100  Topic10  -5.7082  -0.4121
126    really    17.214111  1382.278241  Topic10  -5.6635  -0.6774
735     great    15.236941   704.290332  Topic10  -5.7855  -0.1252
55   gameplay    15.080235   672.744581  Topic10  -5.7959  -0.0897
73      hours    15.018135   719.093070  Topic10  -5.8000  -0.1604

[778 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
5870       4  0.547662         aber
9391       6  0.697206  absolutelyi
500        1  0.295828       absurd
500        4  0.147914       absurd
500        5  0.147914       absurd
...      ...       ...          ...
12128      4  0.547656          く__
12132      4  0.547634           ｰr
12063      5  0.462542     𝗛𝗶𝘀𝗽𝗮𝗻𝗶𝗰
12078      5  0.462499           𝗼𝗻
12085      5  0.462786         𝘃𝗲𝗿𝘆

[1661 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 2, 10, 6, 3, 7, 5, 9, 8])